In [2]:
using Revise
using QuantumClifford.ECC
using QuantumClifford

In [3]:
import Nemo: GF
import LinearAlgebra

In [4]:
# test codes LP04 and LP118 are from https://arxiv.org/pdf/2111.07029

B04 = Dict(
    7 => [ 0 0 0 0; 0 1 2 5; 0 6 3 1],
    9 => [0 0 0 0; 0 1 6 7; 0 4 5 2],
    17 => [0 0 0 0; 0 1 2 11; 0 8 12 13],
    19 => [0 0 0 0; 0 2 6 9; 0 16 7 11]
)

B118 = Dict(
    16 => [0 0 0 0 0; 0 2 4 7 11; 0 3 10 14 15],
    21 => [0 0 0 0 0; 0 4 5 7 17; 0 14 18 12 11],
    30 => [0 0 0 0 0; 0 2 14 24 25; 0 16 11 14 13],
)

LP04 = []
for l in keys(B04)
    A = map(B04[l]) do x
        (PermutationGroupRing(GF(2), l))(cyclic_permutation(x, l))
    end
    push!(LP04, LPCode(LiftedCode(A), LiftedCode(A)))
end

LP118 = []
for l in keys(B118)
    A = map(B118[l]) do x
        (PermutationGroupRing(GF(2), l))(cyclic_permutation(x, l))
    end
    push!(LP118, LPCode(LiftedCode(A), LiftedCode(A)))
end

other_lifted_product_codes = []

# from https://arxiv.org/abs/2202.01702v3

l = 63
R = PermutationGroupRing(GF(2), l)
A = zeros(R, 7,7)
A[LinearAlgebra.diagind(A)] .= R(cyclic_permutation(27, l))
A[LinearAlgebra.diagind(A, -1)] .= R(cyclic_permutation(54, l))
A[LinearAlgebra.diagind(A, 6)] .= R(cyclic_permutation(54, l))
A[LinearAlgebra.diagind(A, -2)] .= R(1)
A[LinearAlgebra.diagind(A, 5)] .= R(1)

B = zeros(R, 1, 1)
B[1,1] = (R(1) + R(cyclic_permutation(1, l)) + R(cyclic_permutation(6, l)))'

push!(other_lifted_product_codes, LPCode(LiftedCode(A), LiftedCode(B)))

1-element Vector{Any}:
 LPCode(PermGroupRingElem[Dict{AbstractAlgebra.Perm{Int64}, Nemo.FqFieldElem}((1,28,55,19,46,10,37)(2,29,56,20,47,11,38)(3,30,57,21,48,12,39)(4,31,58,22,49, … => 1) Dict{AbstractAlgebra.Perm, Nemo.FqFieldElem}() … Dict{AbstractAlgebra.Perm{Int64}, Nemo.FqFieldElem}(() => 1) Dict{AbstractAlgebra.Perm{Int64}, Nemo.FqFieldElem}((1,55,46,37,28,19,10)(2,56,47,38,29,20,11)(3,57,48,39,30,21,12)(4,58,49,40,31, … => 1); Dict{AbstractAlgebra.Perm{Int64}, Nemo.FqFieldElem}((1,55,46,37,28,19,10)(2,56,47,38,29,20,11)(3,57,48,39,30,21,12)(4,58,49,40,31, … => 1) Dict{AbstractAlgebra.Perm{Int64}, Nemo.FqFieldElem}((1,28,55,19,46,10,37)(2,29,56,20,47,11,38)(3,30,57,21,48,12,39)(4,31,58,22,49, … => 1) … Dict{AbstractAlgebra.Perm, Nemo.FqFieldElem}() Dict{AbstractAlgebra.Perm{Int64}, Nemo.FqFieldElem}(() => 1); … ; Dict{AbstractAlgebra.Perm, Nemo.FqFieldElem}() Dict{AbstractAlgebra.Perm, Nemo.FqFieldElem}() … Dict{AbstractAlgebra.Perm{Int64}, Nemo.FqFieldElem}((1,28,55,19,46,10,37)

In [5]:
code = LP04[1]
code_k(code)

19

In [6]:
# naive_encoding_circuit(code)

In [7]:
# naive_syndrome_circuit(code)

In [8]:
# shor_syndrome_circuit(code)

In [9]:
import PyQDecoders

codes = vcat(LP04, LP118, other_lifted_product_codes)

noise = 0.001

setups = [
    CommutationCheckECCSetup(noise),
    NaiveSyndromeECCSetup(noise, 0),
    ShorSyndromeECCSetup(noise, 0),
]
# lifted product codes currently trigger errors in syndrome circuits

for c in codes
    for s in setups
        for d in [c->PyBeliefPropOSDecoder(c, maxiter=10)]
            @show c
            @show s
            nsamples = code_n(c)>400 ? 1000 : 100000
            # take fewer samples for larger codes to save time
            e = evaluate_decoder(d(c), s, nsamples)
            @show e

            @assert max(e...) < noise/4 (c, s, e)
        end
    end
end

c = LPCode(PermGroupRingElem[Dict{AbstractAlgebra.Perm{Int64}, Nemo.FqFieldElem}(() => 1) Dict{AbstractAlgebra.Perm{Int64}, Nemo.FqFieldElem}(() => 1) Dict{AbstractAlgebra.Perm{Int64}, Nemo.FqFieldElem}(() => 1) Dict{AbstractAlgebra.Perm{Int64}, Nemo.FqFieldElem}(() => 1); Dict{AbstractAlgebra.Perm{Int64}, Nemo.FqFieldElem}(() => 1) Dict{AbstractAlgebra.Perm{Int64}, Nemo.FqFieldElem}((1,2,3,4,5,6,7) => 1) Dict{AbstractAlgebra.Perm{Int64}, Nemo.FqFieldElem}((1,3,5,7,2,4,6) => 1) Dict{AbstractAlgebra.Perm{Int64}, Nemo.FqFieldElem}((1,6,4,2,7,5,3) => 1); Dict{AbstractAlgebra.Perm{Int64}, Nemo.FqFieldElem}(() => 1) Dict{AbstractAlgebra.Perm{Int64}, Nemo.FqFieldElem}((1,7,6,5,4,3,2) => 1) Dict{AbstractAlgebra.Perm{Int64}, Nemo.FqFieldElem}((1,4,7,3,6,2,5) => 1) Dict{AbstractAlgebra.Perm{Int64}, Nemo.FqFieldElem}((1,2,3,4,5,6,7) => 1)], PermGroupRingElem[Dict{AbstractAlgebra.Perm{Int64}, Nemo.FqFieldElem}(() => 1) Dict{AbstractAlgebra.Perm{Int64}, Nemo.FqFieldElem}(() => 1) Dict{AbstractAlge

┌ Warning: `faults_matrix` was called on an ECC that has redundant rows (is rank-deficient). `faults_matrix` corrected for that, however this is a frequent source of mistakes and inefficiencies. We advise you remove redundant rows from your ECC.
└ @ QuantumClifford.ECC ~/Documents/Repos/QuantumClifford.jl/src/ecc/ECC.jl:310
┌ Warning: `faults_matrix` was called on an ECC that has redundant rows (is rank-deficient). `faults_matrix` corrected for that, however this is a frequent source of mistakes and inefficiencies. We advise you remove redundant rows from your ECC.
└ @ QuantumClifford.ECC ~/Documents/Repos/QuantumClifford.jl/src/ecc/ECC.jl:310


e = 4.0e-5
c = LPCode(PermGroupRingElem[Dict{AbstractAlgebra.Perm{Int64}, Nemo.FqFieldElem}(() => 1) Dict{AbstractAlgebra.Perm{Int64}, Nemo.FqFieldElem}(() => 1) Dict{AbstractAlgebra.Perm{Int64}, Nemo.FqFieldElem}(() => 1) Dict{AbstractAlgebra.Perm{Int64}, Nemo.FqFieldElem}(() => 1); Dict{AbstractAlgebra.Perm{Int64}, Nemo.FqFieldElem}(() => 1) Dict{AbstractAlgebra.Perm{Int64}, Nemo.FqFieldElem}((1,2,3,4,5,6,7) => 1) Dict{AbstractAlgebra.Perm{Int64}, Nemo.FqFieldElem}((1,3,5,7,2,4,6) => 1) Dict{AbstractAlgebra.Perm{Int64}, Nemo.FqFieldElem}((1,6,4,2,7,5,3) => 1); Dict{AbstractAlgebra.Perm{Int64}, Nemo.FqFieldElem}(() => 1) Dict{AbstractAlgebra.Perm{Int64}, Nemo.FqFieldElem}((1,7,6,5,4,3,2) => 1) Dict{AbstractAlgebra.Perm{Int64}, Nemo.FqFieldElem}((1,4,7,3,6,2,5) => 1) Dict{AbstractAlgebra.Perm{Int64}, Nemo.FqFieldElem}((1,2,3,4,5,6,7) => 1)], PermGroupRingElem[Dict{AbstractAlgebra.Perm{Int64}, Nemo.FqFieldElem}(() => 1) Dict{AbstractAlgebra.Perm{Int64}, Nemo.FqFieldElem}(() => 1) Dict{A

┌ Warning: `faults_matrix` was called on an ECC that has redundant rows (is rank-deficient). `faults_matrix` corrected for that, however this is a frequent source of mistakes and inefficiencies. We advise you remove redundant rows from your ECC.
└ @ QuantumClifford.ECC ~/Documents/Repos/QuantumClifford.jl/src/ecc/ECC.jl:310
┌ Warning: `faults_matrix` was called on an ECC that has redundant rows (is rank-deficient). `faults_matrix` corrected for that, however this is a frequent source of mistakes and inefficiencies. We advise you remove redundant rows from your ECC.
└ @ QuantumClifford.ECC ~/Documents/Repos/QuantumClifford.jl/src/ecc/ECC.jl:310


e = (2.0e-5, 1.0e-5)
c = LPCode(PermGroupRingElem[Dict{AbstractAlgebra.Perm{Int64}, Nemo.FqFieldElem}(() => 1) Dict{AbstractAlgebra.Perm{Int64}, Nemo.FqFieldElem}(() => 1) Dict{AbstractAlgebra.Perm{Int64}, Nemo.FqFieldElem}(() => 1) Dict{AbstractAlgebra.Perm{Int64}, Nemo.FqFieldElem}(() => 1); Dict{AbstractAlgebra.Perm{Int64}, Nemo.FqFieldElem}(() => 1) Dict{AbstractAlgebra.Perm{Int64}, Nemo.FqFieldElem}((1,2,3,4,5,6,7) => 1) Dict{AbstractAlgebra.Perm{Int64}, Nemo.FqFieldElem}((1,3,5,7,2,4,6) => 1) Dict{AbstractAlgebra.Perm{Int64}, Nemo.FqFieldElem}((1,6,4,2,7,5,3) => 1); Dict{AbstractAlgebra.Perm{Int64}, Nemo.FqFieldElem}(() => 1) Dict{AbstractAlgebra.Perm{Int64}, Nemo.FqFieldElem}((1,7,6,5,4,3,2) => 1) Dict{AbstractAlgebra.Perm{Int64}, Nemo.FqFieldElem}((1,4,7,3,6,2,5) => 1) Dict{AbstractAlgebra.Perm{Int64}, Nemo.FqFieldElem}((1,2,3,4,5,6,7) => 1)], PermGroupRingElem[Dict{AbstractAlgebra.Perm{Int64}, Nemo.FqFieldElem}(() => 1) Dict{AbstractAlgebra.Perm{Int64}, Nemo.FqFieldElem}(() =>

...

e = 0.0
c = LPCode(PermGroupRingElem[Dict{AbstractAlgebra.Perm{Int64}, Nemo.FqFieldElem}((1,28,55,19,46,10,37)(2,29,56,20,47,11,38)(3,30,57,21,48,12,39)(4,31,58,22,49, … => 1) Dict{AbstractAlgebra.Perm, Nemo.FqFieldElem}() Dict{AbstractAlgebra.Perm, Nemo.FqFieldElem}() Dict{AbstractAlgebra.Perm, Nemo.FqFieldElem}() Dict{AbstractAlgebra.Perm, Nemo.FqFieldElem}() Dict{AbstractAlgebra.Perm{Int64}, Nemo.FqFieldElem}(() => 1) Dict{AbstractAlgebra.Perm{Int64}, Nemo.FqFieldElem}((1,55,46,37,28,19,10)(2,56,47,38,29,20,11)(3,57,48,39,30,21,12)(4,58,49,40,31, … => 1); Dict{AbstractAlgebra.Perm{Int64}, Nemo.FqFieldElem}((1,55,46,37,28,19,10)(2,56,47,38,29,20,11)(3,57,48,39,30,21,12)(4,58,49,40,31, … => 1) Dict{AbstractAlgebra.Perm{Int64}, Nemo.FqFieldElem}((1,28,55,19,46,10,37)(2,29,56,20,47,11,38)(3,30,57,21,48,12,39)(4,31,58,22,49, … => 1) Dict{AbstractAlgebra.Perm, Nemo.FqFieldElem}() Dict{AbstractAlgebra.Perm, Nemo.FqFieldElem}() Dict{AbstractAlgebra.Perm, Nemo.FqFieldElem}() Dict{AbstractAlg

┌ Warning: `faults_matrix` was called on an ECC that has redundant rows (is rank-deficient). `faults_matrix` corrected for that, however this is a frequent source of mistakes and inefficiencies. We advise you remove redundant rows from your ECC.
└ @ QuantumClifford.ECC ~/Documents/Repos/QuantumClifford.jl/src/ecc/ECC.jl:310
┌ Warning: `faults_matrix` was called on an ECC that has redundant rows (is rank-deficient). `faults_matrix` corrected for that, however this is a frequent source of mistakes and inefficiencies. We advise you remove redundant rows from your ECC.
└ @ QuantumClifford.ECC ~/Documents/Repos/QuantumClifford.jl/src/ecc/ECC.jl:310


e = (0.0, 0.0)
c = LPCode(PermGroupRingElem[Dict{AbstractAlgebra.Perm{Int64}, Nemo.FqFieldElem}((1,28,55,19,46,10,37)(2,29,56,20,47,11,38)(3,30,57,21,48,12,39)(4,31,58,22,49, … => 1) Dict{AbstractAlgebra.Perm, Nemo.FqFieldElem}() Dict{AbstractAlgebra.Perm, Nemo.FqFieldElem}() Dict{AbstractAlgebra.Perm, Nemo.FqFieldElem}() Dict{AbstractAlgebra.Perm, Nemo.FqFieldElem}() Dict{AbstractAlgebra.Perm{Int64}, Nemo.FqFieldElem}(() => 1) Dict{AbstractAlgebra.Perm{Int64}, Nemo.FqFieldElem}((1,55,46,37,28,19,10)(2,56,47,38,29,20,11)(3,57,48,39,30,21,12)(4,58,49,40,31, … => 1); Dict{AbstractAlgebra.Perm{Int64}, Nemo.FqFieldElem}((1,55,46,37,28,19,10)(2,56,47,38,29,20,11)(3,57,48,39,30,21,12)(4,58,49,40,31, … => 1) Dict{AbstractAlgebra.Perm{Int64}, Nemo.FqFieldElem}((1,28,55,19,46,10,37)(2,29,56,20,47,11,38)(3,30,57,21,48,12,39)(4,31,58,22,49, … => 1) Dict{AbstractAlgebra.Perm, Nemo.FqFieldElem}() Dict{AbstractAlgebra.Perm, Nemo.FqFieldElem}() Dict{AbstractAlgebra.Perm, Nemo.FqFieldElem}() Dict{Abst

┌ Warning: `faults_matrix` was called on an ECC that has redundant rows (is rank-deficient). `faults_matrix` corrected for that, however this is a frequent source of mistakes and inefficiencies. We advise you remove redundant rows from your ECC.
└ @ QuantumClifford.ECC ~/Documents/Repos/QuantumClifford.jl/src/ecc/ECC.jl:310
┌ Warning: `faults_matrix` was called on an ECC that has redundant rows (is rank-deficient). `faults_matrix` corrected for that, however this is a frequent source of mistakes and inefficiencies. We advise you remove redundant rows from your ECC.
└ @ QuantumClifford.ECC ~/Documents/Repos/QuantumClifford.jl/src/ecc/ECC.jl:310


e = (0.0, 0.0)


# Deal with promote errors

solved by using `vcat`, as reported in https://github.com/JuliaLang/julia/issues/45454

In [33]:
shor_syndrome_circuit(LP04[3])

(QuantumClifford.AbstractOperation[QuantumClifford.sHadamard(426), QuantumClifford.sCNOT(426,427), QuantumClifford.sCNOT(427,428), QuantumClifford.sCNOT(428,429), QuantumClifford.sCNOT(429,430), QuantumClifford.sCNOT(430,431), QuantumClifford.sCNOT(431,432), QuantumClifford.sHadamard(433), QuantumClifford.sCNOT(433,434), QuantumClifford.sCNOT(434,435)  …  QuantumClifford.sCNOT(3271,3272), QuantumClifford.sCNOT(3272,3273), QuantumClifford.sCNOT(3273,3274), QuantumClifford.sHadamard(3275), QuantumClifford.sCNOT(3275,3276), QuantumClifford.sCNOT(3276,3277), QuantumClifford.sCNOT(3277,3278), QuantumClifford.sCNOT(3278,3279), QuantumClifford.sCNOT(3279,3280), QuantumClifford.sCNOT(3280,3281)], QuantumClifford.AbstractOperation[QuantumClifford.sXCZ(1,426), QuantumClifford.sXCZ(69,427), QuantumClifford.sXCZ(137,428), QuantumClifford.sXCZ(205,429), QuantumClifford.sXCZ(273,430), QuantumClifford.sXCZ(290,431), QuantumClifford.sXCZ(307,432), QuantumClifford.sMRX(426, 1), QuantumClifford.sMRX(427

In [32]:
# naive_encoding_circuit(parity_checks(code))

In [34]:
shor_syndrome_circuit(parity_checks(LP04[3]))

(QuantumClifford.AbstractOperation[QuantumClifford.sHadamard(426), QuantumClifford.sCNOT(426,427), QuantumClifford.sCNOT(427,428), QuantumClifford.sCNOT(428,429), QuantumClifford.sCNOT(429,430), QuantumClifford.sCNOT(430,431), QuantumClifford.sCNOT(431,432), QuantumClifford.sHadamard(433), QuantumClifford.sCNOT(433,434), QuantumClifford.sCNOT(434,435)  …  QuantumClifford.sCNOT(3271,3272), QuantumClifford.sCNOT(3272,3273), QuantumClifford.sCNOT(3273,3274), QuantumClifford.sHadamard(3275), QuantumClifford.sCNOT(3275,3276), QuantumClifford.sCNOT(3276,3277), QuantumClifford.sCNOT(3277,3278), QuantumClifford.sCNOT(3278,3279), QuantumClifford.sCNOT(3279,3280), QuantumClifford.sCNOT(3280,3281)], QuantumClifford.AbstractOperation[QuantumClifford.sXCZ(1,426), QuantumClifford.sXCZ(69,427), QuantumClifford.sXCZ(137,428), QuantumClifford.sXCZ(205,429), QuantumClifford.sXCZ(273,430), QuantumClifford.sXCZ(290,431), QuantumClifford.sXCZ(307,432), QuantumClifford.sMRX(426, 1), QuantumClifford.sMRX(427

In [58]:
function evaluate_decoder_debug(d, nsamples, circuit, syndrome_bits, logical_bits, faults_submatrix)
    frames = pftrajectories(circuit;trajectories=nsamples,threads=true) # bug here
    print("111")
    syndromes = @view pfmeasurements(frames)[:, syndrome_bits]
    measured_faults = @view pfmeasurements(frames)[:, logical_bits]
    guesses = batchdecode(d, syndromes)
    evaluate_guesses(measured_faults, guesses, faults_submatrix)
end

evaluate_decoder_debug (generic function with 1 method)

In [68]:
code = LP04[3]

d = code->PyBeliefPropOSDecoder(code, maxiter=10)
setup = NaiveSyndromeECCSetup(noise, 0)
nsamples= 10

function physical_ECC_circuit_debug(H, setup::NaiveSyndromeECCSetup)
    # syndrome_circ, n_anc, syndrome_bits = naive_syndrome_circuit(remove_redundant_rows(H)) # TODO as reported, this seems to be a bad idea
    syndrome_circ, n_anc, syndrome_bits = naive_syndrome_circuit(H)
    noisy_syndrome_circ = []

    for op in syndrome_circ
        push!(noisy_syndrome_circ, op)
        # for noise_op in add_two_qubit_gate_noise(op, setup.two_qubit_gate_noise)
        #     push!(noisy_syndrome_circ, noise_op)
        # end
    end

    # mem_error_circ = [PauliError(i, setup.mem_noise) for i in 1:175]
    # circ = [mem_error_circ..., noisy_syndrome_circ...]
    circ = noisy_syndrome_circ
    return circ, syndrome_bits, n_anc
end

H = parity_checks(d(code))
n = code_n(H)
k = code_k(H)
O = faults_matrix(H)

physical_noisy_circ, syndrome_bits, n_anc = physical_ECC_circuit_debug(H, setup)
encoding_circ = naive_encoding_circuit(H)
preX = [sHadamard(i) for i in n-k+1:n]

mdH = MixedDestabilizer(H)
logX_circ, _, logX_bits = naive_syndrome_circuit(logicalxview(mdH), n_anc+1, last(syndrome_bits)+1)
logZ_circ, _, logZ_bits = naive_syndrome_circuit(logicalzview(mdH), n_anc+1, last(syndrome_bits)+1)

# pftrajectories([encoding_circ..., physical_noisy_circ..., logZ_circ...])

pftrajectories(vcat(encoding_circ, physical_noisy_circ, logZ_circ))


# # Evaluate the probability for X logical error (the Z-observable part of the faults matrix is used)
# X_error = evaluate_decoder(
#     d(code), nsamples,
#     [encoding_circ..., physical_noisy_circ..., logZ_circ...],
#     syndrome_bits, logZ_bits, O[end÷2+1:end,:])
# # Evaluate the probability for Z logical error (the X-observable part of the faults matrix is used)
# Z_error = evaluate_decoder(
#     d(code), nsamples,
#     [preX..., encoding_circ..., physical_noisy_circ..., logX_circ...],
#     syndrome_bits, logX_bits, O[1:end÷2,:])

# return (X_error, Z_error)

┌ Warning: `faults_matrix` was called on an ECC that has redundant rows (is rank-deficient). `faults_matrix` corrected for that, however this is a frequent source of mistakes and inefficiencies. We advise you remove redundant rows from your ECC.
└ @ QuantumClifford.ECC ~/Documents/Repos/QuantumClifford.jl/src/ecc/ECC.jl:298
┌ Warning: `faults_matrix` was called on an ECC that has redundant rows (is rank-deficient). `faults_matrix` corrected for that, however this is a frequent source of mistakes and inefficiencies. We advise you remove redundant rows from your ECC.
└ @ QuantumClifford.ECC ~/Documents/Repos/QuantumClifford.jl/src/ecc/ECC.jl:298


PauliFrame{Stabilizer{QuantumClifford.Tableau{Vector{UInt8}, LinearAlgebra.Adjoint{UInt64, Matrix{UInt64}}}}, Matrix{Bool}}(Stabilizer 5000×862, Bool[0 0 … 0 0; 0 0 … 0 0; … ; 0 0 … 0 0; 0 0 … 0 0])